# Data ingestion

***This notebook works best with the `conda_python3` on the `ml.t3.xlarge` instance***.

---

In this notebook we download the images corresponding to the slide deck that we uploaded into Amazon S3 in the [1_data_prep.ipynb](./1_data_prep) notebook, convert them into embeddings and then ingest these embeddings into a vector database i.e. [Amazon OpenSearch Service Serverless](https://aws.amazon.com/opensearch-service/features/serverless/).

1. We use the [Anthropic’s Claude 3 Sonnet foundation model](https://aws.amazon.com/about-aws/whats-new/2024/03/anthropics-claude-3-sonnet-model-amazon-bedrock/) available on Bedrock to convert image to text.

1. We then use [Amazon Titan Text Embeddings](https://docs.aws.amazon.com/bedrock/latest/userguide/titan-embedding-models.html) model to convert the text into embeddings.

1. The embeddings are then ingested into OpenSearch Service Serverless using the [Amazon OpenSearch Ingestion](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/ingestion.html) pipeline. We ingest the embeddings into an OpenSearch Serverless index via the OpenSearch Ingestion API.

1. The OpenSearch Service Serverless Collection is created via the AWS CloudFormation stack for this blog post.


## Step 1. Setup

Install the required Python packages and import the relevant files.

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

  Cloning https://github.com/haotian-liu/LLaVA.git (to revision v1.1.1) to /tmp/pip-req-build-fqrp3lh8
  Running command git clone --filter=blob:none --quiet https://github.com/haotian-liu/LLaVA.git /tmp/pip-req-build-fqrp3lh8
  Running command git checkout -q 1619889c712e347be1cb4f78ec66e7cf414ac1a6
  Resolved https://github.com/haotian-liu/LLaVA.git to commit 1619889c712e347be1cb4f78ec66e7cf414ac1a6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os
import time
import glob
import json
import time
import boto3
import codecs
import base64
import logging
import requests
import botocore
import sagemaker
import numpy as np
import globals as g
from typing import List
from pathlib import Path
from requests_auth_aws_sigv4 import AWSSigV4
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from utils import get_cfn_outputs, get_bucket_name, download_image_files_from_s3, get_text_embedding

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
bucket_name: str = get_bucket_name(g.CFN_STACK_NAME)
s3 = boto3.client('s3')

In [4]:
sagemaker_session = sagemaker.Session()
sm_client = sagemaker_session.sagemaker_client
sm_runtime_client = sagemaker_session.sagemaker_runtime_client

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [5]:
bedrock = boto3.client(service_name="bedrock-runtime", region_name=g.AWS_REGION, endpoint_url=g.FMC_URL)

In [6]:
# TODO: read pipeline endpoint and index name from CFN outputs
outputs = get_cfn_outputs(g.CFN_STACK_NAME)
host = outputs['MultimodalCollectionEndpoint'].split('//')[1]
index_name = outputs['OpenSearchIndexName']
logger.info(f"opensearchhost={host}, index={index_name}")

# osi_endpoint = outputs['OpenSearchPipelineEndpoint']
osi_endpoint = "https://test-pipeline-4-5tdu5j2gt7yf6kzpuqqt74jcxm.us-east-1.osis.amazonaws.com/data/ingest"

[2024-03-15 19:05:00,198] p18720 {2967827658.py:5} INFO - opensearchhost=dkubp87d98zgk2ir1t85.us-east-1.aoss.amazonaws.com, index=slides


We use the OpenSearch client to create an index.

In [7]:
session = boto3.Session()
credentials = session.get_credentials()
auth = AWSV4SignerAuth(credentials, g.AWS_REGION, g.OS_SERVICE)

os_client = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)

[2024-03-15 19:05:00,258] p18720 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [8]:
index_body = """
{
  "settings": {
    "index.knn": true
  },
  "mappings": {
    "properties": {
      "vector_embedding": {
        "type": "knn_vector",
        "dimension": 1536,
        "method": {
          "name": "hnsw",
          "engine": "nmslib",
          "parameters": {}
        }
      },
      "image_path": {
        "type": "text"
      },
      "slide_text": {
        "type": "text"
      },
      "slide_number": {
        "type": "text"
      },
       "metadata": { 
        "properties" :
          {
            "filename" : {
              "type" : "text"
            },
            "desc":{
              "type": "text"
            }
          }
      }
    }
  }
}
"""

# We would get an index already exists exception if the index already exists, and that is fine.
index_body = json.loads(index_body)
try:
    response = os_client.indices.create(index_name, body=index_body)
    logger.info(f"response received for the create index -> {response}")
except Exception as e:
    logger.error(f"error in creating index={index_name}, exception={e}")

[2024-03-15 19:05:00,710] p18720 {base.py:259} INFO - PUT https://dkubp87d98zgk2ir1t85.us-east-1.aoss.amazonaws.com:443/slides [status:200 request:0.442s]
[2024-03-15 19:05:00,711] p18720 {952156850.py:46} INFO - response received for the create index -> {'acknowledged': True, 'shards_acknowledged': True, 'index': 'slides'}


## Step 2. Download the images files from S3 and convert to Base64

Now we download the image files from the S3 bucket. Once downloaded these files are converted into [Base64](https://en.wikipedia.org/wiki/Base64) encoding so that we can create embeddings from the images.

In [9]:
# download images from S3, we would be converting these to embeddings
image_files: List = download_image_files_from_s3(bucket_name, g.BUCKET_IMG_PREFIX, g.IMAGE_DIR, g.IMAGE_FILE_EXTN)
logger.info(f"downloaded {len(image_files)} from s3")

[2024-03-15 19:05:00,893] p18720 {utils.py:35} INFO - downloaded multimodal-bucket-563851014557/multimodal/img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_1.jpg to img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_1.jpg
[2024-03-15 19:05:00,943] p18720 {utils.py:35} INFO - downloaded multimodal-bucket-563851014557/multimodal/img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_10.jpg to img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_10.jpg
[2024-03-15 19:05:00,998] p18720 {utils.py:35} INFO - downloaded multimodal-bucket-563851014557/multimodal/img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_11.jpg to img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_11.jpg
[2024-03-15 19:05:01,069] p18720 {utils.py:35} INFO - downloaded multimodal-bucket-563851014557/multimodal/img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_12.jpg to img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_12.jpg
[2024-03-15 19:05:01,121] p18720 {utils.py:35} INFO - downloaded multimodal-bucket-563851014557/multimodal

Convert jpg files into Base64.

In [10]:
def encode_image_to_base64(image_file_path: str) -> str:
    with open(image_file_path, "rb") as image_file:
        b64_image = base64.b64encode(image_file.read()).decode('utf8')
        b64_image_path = os.path.join(g.B64_ENCODED_IMAGES_DIR, f"{Path(image_file_path).stem}.b64")
        with open(b64_image_path, "wb") as b64_image_file:
            b64_image_file.write(bytes(b64_image, 'utf-8'))
    return b64_image_path

## Step 3. Get embeddings for the base64 encoded images

Now we are ready to use Amazon Bedrock via the  Anthropic’s Claude 3 Sonnet foundation model and Amazon Titan Text Embeddings model to convert the base64 version of the images into embeddings. We ingest embeddings into the pipeline using the [requests](https://pypi.org/project/requests/) HTTP library

You must sign all HTTP requests to the pipeline using [Signature Version 4](https://docs.aws.amazon.com/general/latest/gr/signature-version-4.html).

In [11]:
def get_img_desc(image_file_path: str, prompt: str):
    # read the file, MAX image size supported is 2048 * 2048 pixels
    with open(image_file_path, "rb") as image_file:
        input_image_b64 = image_file.read().decode('utf-8')
  
    body = json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 1000,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/jpeg",
                                "data": input_image_b64
                            },
                        },
                        {"type": "text", "text": prompt},
                    ],
                }
            ],
        }
    )
    
    response = bedrock.invoke_model(
        modelId=g.CLAUDE_MODEL_ID,
        body=body
    )

    resp_body = json.loads(response['body'].read().decode("utf-8"))
    resp_text = resp_body['content'][0]['text'].replace('"', "'")

    return resp_text

In [12]:
# download images from S3, we would be converting these to embeddings
image_files: List = download_image_files_from_s3(bucket_name, g.BUCKET_IMG_PREFIX, g.IMAGE_DIR, g.IMAGE_FILE_EXTN)
logger.info(f"downloaded {len(image_files)} from s3")

[2024-03-15 19:05:03,326] p18720 {utils.py:35} INFO - downloaded multimodal-bucket-563851014557/multimodal/img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_1.jpg to img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_1.jpg
[2024-03-15 19:05:03,390] p18720 {utils.py:35} INFO - downloaded multimodal-bucket-563851014557/multimodal/img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_10.jpg to img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_10.jpg
[2024-03-15 19:05:03,426] p18720 {utils.py:35} INFO - downloaded multimodal-bucket-563851014557/multimodal/img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_11.jpg to img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_11.jpg
[2024-03-15 19:05:03,502] p18720 {utils.py:35} INFO - downloaded multimodal-bucket-563851014557/multimodal/img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_12.jpg to img/CMP301_TrainDeploy_E1_20230607_SPEdited_image_12.jpg
[2024-03-15 19:05:03,548] p18720 {utils.py:35} INFO - downloaded multimodal-bucket-563851014557/multimodal

In [13]:
os.makedirs(g.B64_ENCODED_IMAGES_DIR, exist_ok=True)
file_list: List = glob.glob(os.path.join(g.IMAGE_DIR, f"*{g.IMAGE_FILE_EXTN}"))
logger.info(f"there are {len(file_list)} files in the {g.IMAGE_DIR} directory for conversion to base64")

# convert each file to base64 and store the base64 in a new file
b64_image_file_list = list(map(encode_image_to_base64, file_list))
logger.info(f"base64 conversion done, there are {len(b64_image_file_list)} base64 encoded files")

[2024-03-15 19:05:04,873] p18720 {4025043965.py:3} INFO - there are 31 files in the img directory for conversion to base64
[2024-03-15 19:05:04,891] p18720 {4025043965.py:7} INFO - base64 conversion done, there are 31 base64 encoded files


In [14]:
# prompt = "Describe the image in detail, include titles and numbers."

prompt = """
Describe the slide you see in detail. Describe any images, tables, and charts you see in detail with accurate numbers be descriptive about all figures/charts including column/row names. Your response should be extremely detailed and data oriented. Only give the data/numbers that are clearly visible in the image and DO NOT mention anything if it is not in the image or if it is blurry. Be completely accurate"
"""

print(prompt)


Describe the slide you see in detail. Describe any images, tables, and charts you see in detail with accurate numbers be descriptive about all figures/charts including column/row names. Your response should be extremely detailed and data oriented. Only give the data/numbers that are clearly visible in the image and DO NOT mention anything if it is not in the image or if it is blurry. Be completely accurate"



Loop through b64 images to 1/get image desc from Claude3, 2/get embedding from Titan text.
Call OSI pipeline API to ingest embedding.

In [15]:
slide_number = 1
for image_file_path in b64_image_file_list:  
    logger.info(f"going to convert {image_file_path} into embeddings")
    resp_text = get_img_desc(image_file_path, prompt)
    embedding = get_text_embedding(bedrock, resp_text)

    input_image_s3 = f"s3://{bucket_name}/{g.BUCKET_IMG_PREFIX}/{Path(image_file_path).stem}{g.IMAGE_FILE_EXTN}"
    obj_name = f"{Path(image_file_path).stem}{g.IMAGE_FILE_EXTN}"
    
    
    data = json.dumps([{
        "image_path": input_image_s3, 
        "slide_text": resp_text, 
        "slide_number": slide_number, 
        "metadata": {
            "filename": obj_name, 
            "desc": "" 
        }, 
        "vector_embedding": embedding
    }])

    r = requests.request(
        method='POST', 
        url=osi_endpoint, 
        data=data,
        auth=AWSSigV4('osis'))

    logger.info("Ingesting data into pipeline")
    logger.info(f"Response: {slide_number} - {r.text}")
    slide_number = slide_number + 1

[2024-03-15 19:05:04,911] p18720 {3505597840.py:3} INFO - going to convert img/b64_images/CMP301_TrainDeploy_E1_20230607_SPEdited_image_31.b64 into embeddings
[2024-03-15 19:05:13,547] p18720 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
[2024-03-15 19:05:13,877] p18720 {3505597840.py:28} INFO - Ingesting data into pipeline
[2024-03-15 19:05:13,878] p18720 {3505597840.py:29} INFO - Response: 1 - 200 OK
[2024-03-15 19:05:13,878] p18720 {3505597840.py:3} INFO - going to convert img/b64_images/CMP301_TrainDeploy_E1_20230607_SPEdited_image_24.b64 into embeddings
[2024-03-15 19:05:19,756] p18720 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
[2024-03-15 19:05:19,913] p18720 {3505597840.py:28} INFO - Ingesting data into pipeline
[2024-03-15 19:05:19,914] p18720 {3505597840.py:29} INFO - Response: 2 - 200 OK
[2024-03-15 19:05:19,916] p18720 {3505597840.py:3} INFO - going to convert img/b64_i